In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import librosa
import scipy

import tensorflow as tf
from tensorflow.python.platform import gfile

import random as rn
np.random.seed(42)
rn.seed(12345)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(1234)

PATH_TRAIN='/home/arpith/Desktop/nectec/sounds/train'
LABEL_TO_INDEX_MAP={}

###########return classes and assign number to it in training folder which has sound files#############################
def init(path):
    labels=os.listdir(path) #######list of sub directories in the path
    index=0
    for label in labels:
        LABEL_TO_INDEX_MAP[label]=index   #### initial sub directory name zeroo
        index +=1                         #### increments and assigns next number for next sub directory
    global NUM_LABELS
    NUM_LABELS=len(LABEL_TO_INDEX_MAP)
    return LABEL_TO_INDEX_MAP
    
######################encoding those labels #######################################
def one_hot_encoding(label):
    encoding=[0]*len(LABEL_TO_INDEX_MAP) ############ assigns zeroes initially for length of LABEL_TO_INDEX_MAP (2 places)
    encoding[LABEL_TO_INDEX_MAP[label]]=1  ########### assigns 1 for the presence of the label from LABEL_TO_INDEX_MAP list
    return encoding


##################### FEATURE EXTRACTION ###################################################################################
##################### variance and rms of amplitude(db)############################
def get_amp_var_rms(wave_path):
    fs_rate, signal = scipy.io.wavfile.read(wave_path)
    signal=librosa.amplitude_to_db(signal) ####### changing freq to amplitude in decibels#####
    variance=np.var(signal) ######## get variance value of the amplitude values
    rms = np.sqrt(np.mean(signal**2))  ########## get rms value of amplitude values
    return variance,rms

#################### centroid of spectrum formed by stft####################
def get_cent(wave_path):
    y, sr = librosa.load(wave_path)
    S, phase = librosa.magphase(librosa.stft(y))## ############# librosa.stft performs fourier transform on signals
    #### librosa.magphase returns magnitude and phase of complex values extracted from librosa.stft
    cent=np.mean(librosa.feature.spectral_centroid(y=y, sr=sr, S=None, n_fft=4410, hop_length=2205,
                      freq=None).T,axis=0) ### compute spectral centroid of those magnitude values
    

    rolloff=np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, S=None, n_fft=4410, hop_length=2205, freq=None, roll_percent=0.85).T)
    contrast = np.mean(librosa.feature.spectral_contrast(y=y,n_fft=4410,S=None,hop_length=2205).T,axis=0)#3
    mfccs=np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20,n_fft=4410,hop_length=2205,fmin= 50,n_mels=256).T,axis=0) #1

    return mfccs,cent,contrast,rolloff

def get_others(wave_path):
    y, sr = librosa.load(wave_path)   
    stft = np.abs(librosa.stft(y))
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr, S=None, norm=np.inf, n_fft=4410,
                hop_length=2205, tuning=None).T,axis=0)#
    zero_crossing_rate=np.mean(librosa.feature.zero_crossing_rate(y,frame_length=4410,hop_length=2205).T,axis=0)#5
    spectral_flatness=np.mean(librosa.feature.spectral_flatness(y=y,n_fft=4410,S=None,hop_length=2205).T,axis=0)#6

    rmse=np.mean(librosa.feature.rmse(y=y,frame_length=4410,S=None,hop_length=2205).T,axis=0)
    return chroma,zero_crossing_rate,spectral_flatness,rmse



################### extracting mfcc and encode the label #########################
# def get_batch(batch_size,path):    
path=os.path.join(PATH_TEST,"*",'*.wav')  ######## files having path ending with wav
waves=gfile.Glob(path) ### collects all the files existing in the folder(including subdirectories having file ended with .wav)


#########initialze arrays for each feature###############################
X1=[]
Y=[]
X2=[]
X3=[]
X4=[]
X5=[]
X6=[]
X7=[]
X8=[]
X9=[]
X10=[]
X11=[]
Z=[]
# mfcc,chroma,melspectrogram and spectral contrast
# random.shuffle(waves) #### shuffle the  files( mix of both good and bad files)################

##########get features from each wav file##################################
for wave_path in waves:
    _,label=os.path.split(os.path.dirname(wave_path)) ######splits the subdirectory name from path ##########
    #### _= left out path, label=name of sub directory cutted of from path
    first,second,third,tenth=(get_others(wave_path))  
    fourth,fifth=get_amp_var_rms(wave_path)
    sixth,seventh,eight,nineth=get_cent(wave_path)
#     sixth=get_cent(wave_path)
    X1.append(first)  ### chroma
    X2.append(second) ###  zero
    X3.append(third)  ### spec flatness
    X4.append(fourth) ### amp vars
    X5.append(fifth)  ### amp rms
    X6.append(sixth)  ### MFCC
    X7.append(seventh) ### centroid
    X8.append(eight)    ### contrast  
    X9.append(nineth)  ### rolloff
    X10.append(tenth)  ## rmse
   
    Y.append(label)

########### encode the target values of each wav file##################################
def target(arr):
    yev=[]
    for x in arr:
        if x=='good':
            yev.append(0)
        else:
            yev.append(1)
    return yev
Z=target(Y)

######################################## PRE-PROCESSING FEATURES FOR TRAINING ####################################
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
scale=StandardScaler()

x1=np.array(X1)
x2=np.array(X2)
x3=np.array(X3)
four=np.array(X4)
x4=np.reshape(four, (-1, 1))
five=np.array(X5)
x5=np.reshape(five, (-1, 1))
x6=np.array(X6)

x7=np.array(X7)
x8=np.array(X8)

nine=np.array(X9)
x9=np.reshape(nine, (-1, 1))
x10=np.array(X10)

prenormfeat=np.hstack([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10])

normfeat=scale.fit_transform(prenormfeat)
minmaxfeat=scaler.fit_transform(prenormfeat)

Z=np.array(Z)


##################################### normality tests for both min-max and normalizied features#######################
from matplotlib import pyplot

print("histogram data by min-max")
pyplot.hist(minmaxfeat)
pyplot.show()
print("histogram data by standard scaler")
pyplot.hist(normfeat)
pyplot.show()

from statsmodels.graphics.gofplots import qqplot
print("qqplot data by min-max")
qqplot(minmaxfeat, line='s')
pyplot.show()

print("qqplot data by standard scaler")
qqplot(normfeat, line='s')
pyplot.show()


from scipy.stats import shapiro
print("shapiro data by min-max")
stat, p = shapiro(minmaxfeat)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Sample looks Gaussian (fail to reject H0)')
else:
	print('Sample does not look Gaussian (reject H0)')

print("shapiro data by standard scaler")   
stat, p = shapiro(normfeat)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Sample looks Gaussian (fail to reject H0)')
else:
	print('Sample does not look Gaussian (reject H0)')
    

###############################FEATURE SELECTION ##############################################################
##################### select best features using information gain ################
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
   
selector = SelectKBest(mutual_info_classif, k=46)
X_train_clean = selector.fit_transform(normfeat,Z)

mask = selector.get_support() #list of booleans
new_features = [] # The list of your K best features
names=['chroma1','chroma2','chroma3','chroma4','chroma5','chroma6','chroma7','chroma8','chroma9','chroma10','chroma11','chroma12',
      'zero-cross','spectral-flatness','ampvar','amprms',
       'mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15',
       'mfcc16','mfcc17','mfcc18','mfcc19','mfcc20',
      'cent','contrast1','contrast2','contrast3','contrast4','contrast5','contrast6','contrast7','rolloff','rmse']
for bool, feature in zip(mask, names):
    if bool:
        new_features.append(feature)
        
#################################applying principal component anaysis#################

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA().fit(X_train_clean)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

## finally selected 30 features using principal component analysise, because PCA is also good for selecting features #############
from sklearn import decomposition
pca = decomposition.PCA(n_components=30)
PCAfeatures=pca.fit_transform(X_train_clean)

######################### MODEL TRAINING USING SVM #########################################################################
###SVM GridSearchCV##########################
from sklearn.svm import SVC
tuned_parameters = [{'kernel': ['rbf','linear'], 'gamma': [1e-3, 1e-4, 1e-5 , 1e-6 ,1e-7,1e-8] 
                     ,'C': [1, 5 , 10 , 100, 1000,10000,100000,1000000]}]
# grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

clf = GridSearchCV(SVC(), tuned_parameters, cv=10,
                       scoring='accuracy')
X=PCAfeatures
# X_train_clean
clf.fit(X, Z)
print(clf.best_params_)
print(clf.best_score_)
